# EasyBudget Ingest (Jupyter Edition)
เครื่องมือนี้ทำให้ **กดรันได้ใน Notebook** ไม่ต้องใช้ CMD  
เป้าหมาย: อ่านทุกไฟล์ในโฟลเดอร์ `data/` → ตรวจจับหัวตารางอัตโนมัติ → แม็ปคอลัมน์ตาม Data Dictionary → รวมเป็น `out/clean_data.xlsx` พร้อมรายงานครอบคลุม

> ถ้าได้เฉพาะ "เฮดดิ้ง" แปลว่า **หัวตารางยังจับไม่ตรง/คอลัมน์ยังไม่แม็ป** ⇒ กรอกคำพ้อง/regex เพิ่มใน `config/datadic.xlsx` หรือปรับพารามิเตอร์ใน CONFIG แล้วรันใหม่


## 1) CONFIG – ตั้งค่าครั้งเดียวแล้วรัน

In [36]:

from pathlib import Path

# === Path settings ===
PROJECT_DIR = Path.cwd()  # สามารถเปลี่ยนเป็น Path("D:/YourProject") ถ้าต้องการ
INPUT_DIR   = PROJECT_DIR / "data"     # วางไฟล์ Excel ทั้งหมดไว้ในโฟลเดอร์นี้
OUTPUT_DIR  = PROJECT_DIR / "out"
CONFIG_DIR  = PROJECT_DIR / "config"
DATA_DICT   = CONFIG_DIR / "datadic.xlsx"  # ถ้าไม่มี ให้คัดลอกไฟล์ของกิฟมาใส่

# === Behavior parameters ===
HEADER_SCAN_ROWS = 50     # สแกนกี่แถวแรกเพื่อหาระดับหัวตารางจริง
HEADER_MIN_HITS  = 2      # คะแนนขั้นต่ำในการถือว่าเป็น "หัวตาราง"
MAX_SHEETS       = None   # จำกัดจำนวนชีตต่อไฟล์ (None = ทั้งหมด)
WRITE_MODE       = "xlsx" # 'xlsx' หรือ 'csv'

# Standard schema fields we expect to see (ปรับ/เพิ่มได้)
STANDARD_FIELDS = [
    "year_fy", "agency_code", "agency_name", "province", "plan", "output",
    "activity_code", "activity", "budget_group", "expense_type", "sub_category",
    "commitment_type", "cap_ncap", "item_code", "item_name", "amount",
    # ====== ใหม่จาก flags ======
    "pp_code", "output_kind",            # 5=โครงการ, 3=ผลผลิต → output_kind แปลข้อความ
    "land_building",                     # objc_700_flag
    "asset_band",                        # cpu_flag
    "subsidy_type",                      # subsidy_flag
    "is_commitment",                     # commit_flag (0/1 → True/False)
    "flagship_name",                     # ชื่อ flagship
]


# Output file names
CLEAN_FILE   = OUTPUT_DIR / "clean_data.xlsx"
COVERAGE_XLS = OUTPUT_DIR / "mapping_coverage.xlsx"
UNMAPPED_XLS = OUTPUT_DIR / "unmapped_columns.xlsx"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print("PROJECT_DIR:", PROJECT_DIR)
print("INPUT_DIR:", INPUT_DIR)
print("DATA_DICT exists:", DATA_DICT.exists())

EXCLUDE_PATTERNS = [
    "datadic", "dictionary", "data_dict", "national_budget_datadict",
    "~$",   # ไฟล์ชั่วคราวจาก Excel
]



PROJECT_DIR: c:\Users\netthip\OneDrive\เรียน ป.โท\IS\IS_package
INPUT_DIR: c:\Users\netthip\OneDrive\เรียน ป.โท\IS\IS_package\data
DATA_DICT exists: True


## 2) Utilities – ฟังก์ชันช่วยอ่าน dict/หาหัวตาราง/แม็ปคอลัมน์/ทำความสะอาด

In [37]:
# ===== 2) Utilities – helper functions (วางทับทั้งเซลล์นี้) =====
from pathlib import Path
from typing import Dict, List, Tuple, Optional
import re, math, pandas as pd

pd.options.display.max_columns = 200

# ---------- Data Dictionary (flexible) ----------
def load_data_dictionary(xlsx_path: Path) -> Dict[str, dict]:
    if not xlsx_path.exists():
        raise FileNotFoundError(f"Data dictionary not found: {xlsx_path}")
    xls = pd.ExcelFile(xlsx_path)
    df = pd.read_excel(xls, 0)
    df.columns = [str(c).strip() for c in df.columns]

    def norm(s: str):
        return re.sub(r"[\s_]+", "", str(s).strip().lower())

    cols_norm = {norm(c): c for c in df.columns}

    std_candidates = [
        "standardname","standard","stdname","std","field","fieldname",
        "column","columnname","name","ชื่อมาตรฐาน","มาตรฐาน","ชื่อคอลัมน์มาตรฐาน","ฟิลด์มาตรฐาน"
    ]
    syn_candidates = [
        "synonym","synonyms","alias","aka","mapping","map",
        "ชื่อพ้อง","คำพ้อง","ชื่ออื่น","ชื่อที่พบ","หัวข้อพ้อง"
    ]
    rx_candidates  = [
        "regex","pattern","re","patternregex","regexp",
        "แพทเทิร์น","รูปแบบ","เรกเอกซ์","นิพจน์ปรกติ"
    ]

    def pick(cands):
        for key in cols_norm:
            for c in cands:
                if c in key:
                    return cols_norm[key]
        return None

    col_std = pick(std_candidates)
    col_syn = pick(syn_candidates)
    col_rx  = pick(rx_candidates)

    if not col_std:
        raise KeyError(
            "ไม่พบคอลัมน์สำหรับ 'standard_name' ใน datadic.xlsx\n"
            f"หัวคอลัมน์ที่พบ: {list(df.columns)}\n"
            "วิธีแก้: ตั้งชื่อคอลัมน์ให้มีคำว่า 'standard'/'name' หรือ 'ชื่อมาตรฐาน'"
        )

    dictionary = {}
    for _, row in df.iterrows():
        std_raw = row.get(col_std, None)
        if pd.isna(std_raw): 
            continue
        std = str(std_raw).strip()
        if not std: 
            continue

        syns = []
        if col_syn and pd.notna(row.get(col_syn, None)):
            syns = [s.strip() for s in str(row[col_syn]).split("|") if str(s).strip()]

        rxs = []
        if col_rx and pd.notna(row.get(col_rx, None)):
            parts = [s.strip() for s in str(row[col_rx]).split("|") if str(s).strip()]
            for r in parts:
                try:
                    rxs.append(re.compile(r, flags=re.I))
                except re.error:
                    pass

        dictionary.setdefault(std, {"synonyms": set(), "regex": []})
        dictionary[std]["synonyms"].update({std.lower(), *[s.lower() for s in syns]})
        dictionary[std]["regex"].extend(rxs)

    return dictionary

# ---------- Row filters (ตัดแถวสรุป/ป้ายหัว) ----------
ROW_EXCLUDE_PATTERNS = [
    r"^รวม(ทั้งสิ้น|ยอด)?\b", r"^รวม\s", r"^ผลรวม\b", r"^total\b",
    r"^กระทรวง\s*:", r"^หน่วยงาน\s*:", r"^จังหวัด\s*:", r"^หมายเหตุ\b",
    r"^ภาพรวม", r"^สรุป", r"^ตารางสรุป", r"^งบประมาณรวม\b"
]
ROW_EXCLUDE_RX = re.compile("|".join(ROW_EXCLUDE_PATTERNS), flags=re.I)

def is_banner_or_total(row_vals: List[str]) -> bool:
    for v in row_vals[:3]:
        s = str(v or "").strip()
        if not s:
            continue
        if ROW_EXCLUDE_RX.search(s):
            return True
    return False

# ---------- Header scoring / detection ----------
def row_score_as_header(values: List[str], dict_map: Dict[str, dict]) -> int:
    if is_banner_or_total(values):
        return -999
    score = 0
    for v in values:
        vs = str(v).strip().lower()
        if not vs or vs == "nan":
            continue
        for meta in dict_map.values():
            if vs in meta["synonyms"]:
                score += 2
                break
            else:
                for rx in meta["regex"]:
                    if rx.search(vs):
                        score += 1
                        break
    textish = sum(1 for v in values if re.search(r"[ก-๙a-z]", str(v or "").lower()))
    score += textish
    return score

def find_header_row(df: pd.DataFrame, dict_map: Dict[str, dict], scan_rows: int, min_hits: int) -> Optional[int]:
    best_row, best_score = None, -1
    scan = min(scan_rows, len(df))
    for r in range(scan):
        row_vals = list(df.iloc[r].values)
        s = row_score_as_header(row_vals, dict_map)
        if s > best_score:
            best_row, best_score = r, s
    return best_row if (best_row is not None and best_score >= min_hits) else None

def compose_header(df: pd.DataFrame, header_row: int) -> List[str]:
    base = df.iloc[header_row].fillna("").astype(str).tolist()
    merged = base[:]
    if header_row + 1 < len(df):
        nxt = df.iloc[header_row+1].fillna("").astype(str).tolist()
        if sum(1 for x in nxt if x.strip()) >= max(2, len(nxt)//4):
            merged = [f"{a.strip()} | {b.strip()}".strip(" | ") if (a.strip() and b.strip()) else (a or b) for a,b in zip(base, nxt)]
    return [m.strip() for m in merged]

# ---------- Column mapping (exact/regex + contains heuristic) ----------
def build_column_map(headers: List[str], dict_map: Dict[str, dict]) -> Dict[int, Tuple[str, int]]:
    def tokens(s: str):
        parts = re.split(r"[|\-/]+", str(s).lower())
        return [p.strip() for p in parts if p.strip()]

    result: Dict[int, Tuple[str,int]] = {}

    # ชั้น 1-2: exact + regex จาก dict
    for ci, h in enumerate(headers):
        key = str(h).strip().lower()
        best, score = None, -1
        for std, meta in dict_map.items():
            if key in meta["synonyms"]:
                best, score = std, 100
                break
            for rx in meta["regex"]:
                if rx.search(key):
                    if score < 50:
                        best, score = std, 50
                    break
        if best:
            result[ci] = (best, score)

    # ชั้น 3: contains rules + token สำหรับหัวยาว/ประโยคผสม
    CONTAINS_RULES = [
        ("แผนงาน", "plan"),
        ("ผลผลิต", "output"),
        ("โครงการ", "output"),
        ("กิจกรรม", "activity"),
        ("รหัสกิจกรรม", "activity_code"),
        ("รหัสรายการ", "item_code"),
        ("รายการ", "item_name"),
        ("หมวด", "sub_category"),
        ("งบประมาณ", "amount"),
        ("ภาระผูกพัน", "commitment_type"),
        ("ผูกพัน", "commitment_type"),
        ("ปีเดียว", "commitment_type"),
        ("ลงทุน", "cap_ncap"),
        ("ประจำ", "cap_ncap"),
        ("จังหวัด", "province"),
        ("หน่วยงาน", "agency_name"),
    ]
    for ci, h in enumerate(headers):
        if ci in result:
            continue
        tk = tokens(h)
        joined = " ".join(tk)
        matched = None
        for token, std in CONTAINS_RULES:
            if token in joined:
                matched = std
                break
        if matched:
            result[ci] = (matched, 30)
    return result

# ---------- Number parsing / normalizers ----------
def parse_number(val):
    if pd.isna(val):
        return None
    s = str(val).strip()
    if not s:
        return None
    neg = False
    if s.startswith('(') and s.endswith(')'):
        neg = True
        s = s[1:-1]
    s = s.replace(',', '').strip()
    try:
        num = float(s)
        return -num if neg else num
    except:
        return None

def normalize_row(row: dict) -> dict:
    if "amount" in row:
        row["amount"] = parse_number(row["amount"])
    return row

# ---------- Year detection (แนวนอน) ----------
YEAR_RX = re.compile(r"ปี\s*(25\d{2})")
BEFORE_YEAR_RX = re.compile(r"ก่อนปี\s*(25\d{2})")
FUTURE_RX = re.compile(r"ปีต่อ ๆ? ไป|ปีต่อๆไป|และปีต่อ ๆ? ไป")

def detect_year_columns(headers: List[str]) -> List[Tuple[int, int]]:
    year_cols = []
    skip_cols = set()
    for ci, h in enumerate(headers):
        s = str(h)
        m = YEAR_RX.search(s)
        if m:
            year_cols.append((ci, int(m.group(1))))
            continue
        if BEFORE_YEAR_RX.search(s):
            y = int(BEFORE_YEAR_RX.search(s).group(1)) - 1
            year_cols.append((ci, y))
            continue
        if FUTURE_RX.search(s):
            skip_cols.add(ci)
    return [(ci, y) for (ci, y) in year_cols if ci not in skip_cols]

# ---------- Decode flags (PP_CODE, cap_ncap_sitem, pur_commit_flag, ฯลฯ) ----------
def decode_flags(row: dict) -> dict:
    v = (row.get("pp_code") or "").strip()
    if v != "":
        row["output_kind"] = {"5": "โครงการ", "3": "ผลผลิต"}.get(str(v).strip(), None)

    cv = row.get("cap_ncap")
    if cv is not None and str(cv).strip() != "":
        row["cap_ncap"] = {"1": "ลงทุน", "2": "ประจำ"}.get(str(cv).strip(), row["cap_ncap"])

    pv = (row.get("commitment_type") or "").strip().upper()
    if pv != "":
        row["commitment_type"] = {
            "M": "ผูกพันม.23",
            "S": "ผูกพันสัญญา",
            "": "ผูกพันใหม่",
            "NULL": "ผูกพันใหม่",
        }.get(pv, "ไม่ใช่รายการภาระผูกพัน")

    iv = row.get("is_commitment")
    if iv is not None and str(iv).strip() != "":
        row["is_commitment"] = True if str(iv).strip() == "1" else False

    ov = row.get("land_building")
    if ov is not None and str(ov).strip() != "":
        row["land_building"] = {"1": "ที่ดิน", "2": "สิ่งก่อสร้าง"}.get(str(ov).strip(), None)

    av = row.get("asset_band")
    if av is not None and str(av).strip() != "":
        row["asset_band"] = {
            "1": "ครุภัณฑ์<1ล./สิ่งก่อสร้าง<10ล.",
            "2": "ครุภัณฑ์≥1ล./สิ่งก่อสร้าง≥10ล.",
        }.get(str(av).strip(), None)

    sv = row.get("subsidy_type")
    if sv is not None and str(sv).strip() != "":
        row["subsidy_type"] = {"1": "อุดหนุนทั่วไป", "2": "อุดหนุนเฉพาะกิจ"}.get(str(sv).strip(), None)

    return row

print("✅ Utilities loaded. ไปต่อข้อ 3 ได้เลย")


✅ Utilities loaded. ไปต่อข้อ 3 ได้เลย


---
## ใช้งานไม่ออก? เช็คตามนี้ก่อน
1) **ไม่มีแถวข้อมูล มีแต่เฮดดิ้ง** ⇒ ปรับ `HEADER_SCAN_ROWS` สูงขึ้น หรือเพิ่มคำพ้อง/regex ใน `datadic.xlsx` ให้ตรงกับหัวคอลัมน์จริง  
2) **คอลัมน์สำคัญไม่แม็ป** ⇒ เติม `synonym` หรือ `regex` ใน data dict แล้วรันใหม่  
3) **ตัวเลขไม่เข้า** ⇒ ฟิลด์ `amount` ถูก map แล้วหรือยัง? และคอลัมน์เงินมีเครื่องหมายวงเล็บ/คอมมา? (โค้ดรองรับ)  
4) **ไฟล์หลายรูปแบบ** ⇒ วางทั้งหมดใน `data/` ไม่ต้องระบุชื่อไฟล์เป็นรายตัว เครื่องมือจะสแกนอัตโนมัติ


## 3) Processor – อ่านทุกไฟล์/ทุกชีต แล้วรวมผลลัพธ์

In [38]:
# fallback ถ้าไม่ได้ประกาศจาก CONFIG
try:
    EXCLUDE_PATTERNS
except NameError:
    EXCLUDE_PATTERNS = [
        "datadic", "dictionary", "data_dict", "national_budget_datadict", "~$"
    ]


In [39]:
# ===== 3) Processor – อ่านทุกไฟล์/ทุกชีต แล้วรวมผลลัพธ์ (วางทับทั้งเซลล์นี้) =====
import pandas as pd
from pathlib import Path

def process_workbook(path: Path, dict_map, max_sheets=None):
    records = []
    coverage = []
    unmapped_rows = []

    # ป้องกันไฟล์ถูกล็อก/เปิดไม่ได้
    try:
        xls = pd.ExcelFile(path)
    except PermissionError as e:
        coverage.append({
            "file": path.name, "sheet": None, "status": "permission_denied",
            "mapped_cols": 0, "total_cols": 0, "header_row": None, "error": str(e)
        })
        return records, coverage, unmapped_rows
    except Exception as e:
        coverage.append({
            "file": path.name, "sheet": None, "status": "open_failed",
            "mapped_cols": 0, "total_cols": 0, "header_row": None, "error": str(e)[:200]
        })
        return records, coverage, unmapped_rows

    sheets = xls.sheet_names[:max_sheets] if max_sheets else xls.sheet_names

    for sh in sheets:
        raw = pd.read_excel(xls, sh, header=None, dtype=str)

        # หาแถวหัวตารางจริง
        hdr_row = find_header_row(raw, dict_map, HEADER_SCAN_ROWS, HEADER_MIN_HITS)
        if hdr_row is None:
            coverage.append({
                "file": path.name, "sheet": sh, "status": "header_not_found",
                "mapped_cols": 0, "total_cols": raw.shape[1], "header_row": None
            })
            continue

        headers = compose_header(raw, hdr_row)
        data = raw.iloc[hdr_row+1:].reset_index(drop=True).copy()
        data.columns = headers[:data.shape[1]]

        # แม็ปคอลัมน์
        col_map = build_column_map(headers, dict_map)
        mapped_cols = len(col_map)
        coverage.append({
            "file": path.name, "sheet": sh,
            "status": "ok" if mapped_cols else "no_columns_mapped",
            "mapped_cols": mapped_cols, "total_cols": len(headers),
            "header_row": int(hdr_row)
        })

        if not mapped_cols:
            unmapped_rows.append({
                "file": path.name, "sheet": sh,
                "headers_snapshot": "|".join(headers)
            })
            continue

        # --- สร้างแถวผลลัพธ์ (รองรับปีแนวนอน + ตัดแถวรวม/ป้ายหัว) ---
        year_cols = detect_year_columns(headers)   # [(col_idx, 2569), ...]
        for _, r in data.iterrows():
            # ตัดแถวสรุป/ป้ายหัวออกไปก่อน
            first_text = r.iloc[0] if len(r) else ""
            if is_banner_or_total([first_text]):
                continue

            if year_cols:
                # มีคอลัมน์ปีแนวนอน → แตกเป็นหลายแถว long format
                base = {sf: None for sf in STANDARD_FIELDS}
                for ci, (std, _) in col_map.items():
                    if ci < len(r) and std != "amount":
                        base[std] = r.iloc[ci]
                base = normalize_row(base)
                base = decode_flags(base)

                for ci, year in year_cols:
                    val = parse_number(r.iloc[ci] if ci < len(r) else None)
                    if val is None:
                        continue
                    rec = dict(base)
                    rec["year_fy"] = year
                    rec["amount"]  = val
                    if any(v not in (None, "", "nan") for v in rec.values()):
                        records.append({**rec, "_file": path.name, "_sheet": sh})
            else:
                # ไม่มีคอลัมน์ปี → อ่านแบบเดิม
                rec = {sf: None for sf in STANDARD_FIELDS}
                for ci, (std, _) in col_map.items():
                    if ci < len(r):
                        rec[std] = r.iloc[ci]
                rec = normalize_row(rec)
                rec = decode_flags(rec)
                if any(v not in (None, "", "nan") for v in rec.values()):
                    records.append({**rec, "_file": path.name, "_sheet": sh})

    return records, coverage, unmapped_rows

# รวมไฟล์ที่จะประมวลผล (กรองไฟล์ที่ไม่ใช่ข้อมูลดิบออก)
all_candidates = list(Path(INPUT_DIR).glob("**/*.xls*"))
paths = [
    p for p in sorted(all_candidates)
    if not any(tok in p.name.lower() for tok in EXCLUDE_PATTERNS)
    and not p.name.startswith("~$")
]

dict_map = load_data_dictionary(DATA_DICT)
all_records, all_cov, all_unmapped = [], [], []

if not paths:
    print("⚠️ ยังไม่มีไฟล์ในโฟลเดอร์ data/ - ลองวางไฟล์ Excel แล้วรันใหม่")
else:
    for p in paths:
        r, c, u = process_workbook(p, dict_map, MAX_SHEETS)
        all_records.extend(r); all_cov.extend(c); all_unmapped.extend(u)
    print(f"Processed files: {len(paths)}  → rows: {len(all_records)}")


Processed files: 17  → rows: 83890


## 4) Save Outputs – clean_data + coverage + unmapped

In [40]:

import pandas as pd

df_clean = pd.DataFrame(all_records)
df_cov   = pd.DataFrame(all_cov)
df_unmap = pd.DataFrame(all_unmapped)

if WRITE_MODE == "xlsx":
    with pd.ExcelWriter(CLEAN_FILE, engine="openpyxl") as xw:
        (df_clean if len(df_clean) else pd.DataFrame(columns=STANDARD_FIELDS)).to_excel(xw, index=False, sheet_name="clean")
    with pd.ExcelWriter(COVERAGE_XLS, engine="openpyxl") as xw:
        (df_cov if len(df_cov) else pd.DataFrame(columns=["file","sheet","status","mapped_cols","total_cols","header_row"])).to_excel(xw, index=False, sheet_name="coverage")
    with pd.ExcelWriter(UNMAPPED_XLS, engine="openpyxl") as xw:
        (df_unmap if len(df_unmap) else pd.DataFrame(columns=["file","sheet","headers_snapshot"])).to_excel(xw, index=False, sheet_name="unmapped")
else:
    df_clean.to_csv(OUTPUT_DIR / "clean_data.csv", index=False, encoding="utf-8-sig")
    df_cov.to_csv(OUTPUT_DIR / "mapping_coverage.csv", index=False, encoding="utf-8-sig")
    df_unmap.to_csv(OUTPUT_DIR / "unmapped_columns.csv", index=False, encoding="utf-8-sig")

print("✅ Done. Files in:", OUTPUT_DIR)


✅ Done. Files in: c:\Users\netthip\OneDrive\เรียน ป.โท\IS\IS_package\out


---
## ใช้งานไม่ออก? เช็คตามนี้ก่อน
1) **ไม่มีแถวข้อมูล มีแต่เฮดดิ้ง** ⇒ ปรับ `HEADER_SCAN_ROWS` สูงขึ้น หรือเพิ่มคำพ้อง/regex ใน `datadic.xlsx` ให้ตรงกับหัวคอลัมน์จริง  
2) **คอลัมน์สำคัญไม่แม็ป** ⇒ เติม `synonym` หรือ `regex` ใน data dict แล้วรันใหม่  
3) **ตัวเลขไม่เข้า** ⇒ ฟิลด์ `amount` ถูก map แล้วหรือยัง? และคอลัมน์เงินมีเครื่องหมายวงเล็บ/คอมมา? (โค้ดรองรับ)  
4) **ไฟล์หลายรูปแบบ** ⇒ วางทั้งหมดใน `data/` ไม่ต้องระบุชื่อไฟล์เป็นรายตัว เครื่องมือจะสแกนอัตโนมัติ



## โค้ด QC (เพิ่มเป็น “เซลล์ใหม่” ใต้ข้อ 4 ได้เลย)---



In [41]:
# 1) สรุปภาพรวมไฟล์/ชีตที่อ่านได้ 

import pandas as pd, os
from pathlib import Path

out = OUTPUT_DIR  # ใช้ตัวแปรจาก CONFIG
cov = pd.read_excel(out / "mapping_coverage.xlsx")

print("สถานะรวม:")
print(cov["status"].value_counts(), "\n")

print("10 รายการที่แม็ปคอลัมน์ได้น้อยที่สุด:")
print(cov.sort_values(["mapped_cols","total_cols"]).head(10)[["file","sheet","mapped_cols","total_cols","status"]])


สถานะรวม:
status
ok                   58
header_not_found      4
no_columns_mapped     4
Name: count, dtype: int64 

10 รายการที่แม็ปคอลัมน์ได้น้อยที่สุด:
                             file            sheet  mapped_cols  total_cols  \
2   1001_รายงานคำขอเบื้องต้น.xlsx           sheet2            0           0   
28                   1017_P1.xlsx           Sheet1            0           0   
63         รายการบรรทัดยาว ๆ.xlsx  1749618491_5178            0           1   
65             แบบฟอร์ม_2569.xlsx      5รายละเอียด            0           1   
27                   1017_P1.xlsx         หมายเหตุ            0           2   
43        9012_ทะเบียนรายการ.xlsx            Mask1            0           5   
29                   1017_P1.xlsx            mask1            0          16   
36                   1017_P1.xlsx     Mask1_remake            0          16   
4   1001_รายงานคำขอเบื้องต้น.xlsx            mask2            1           5   
42        9012_ทะเบียนรายการ.xlsx           Sheet1     

In [42]:
# 2) เช็ก null-rate ของคอลัมน์สำคัญ
clean = pd.read_excel(out / "clean_data.xlsx")
focus_cols = ["item_name","amount","year_fy","agency_name","province","budget_group","expense_type"]
focus_cols = [c for c in focus_cols if c in clean.columns]

print("จำนวนแถว:", len(clean))
print("อัตรา null โดยคอลัมน์สำคัญ:")
print((clean[focus_cols].isna().mean().sort_values(ascending=False) * 100).round(1).astype(str) + "%")


จำนวนแถว: 83890
อัตรา null โดยคอลัมน์สำคัญ:
expense_type    100.0%
budget_group    100.0%
province        100.0%
agency_name     100.0%
year_fy          97.3%
amount           41.1%
item_name        11.0%
dtype: object


In [43]:
# 3) ตัวเลขเข้าไหม (จำนวนบรรทัดที่ amount ไม่ใช่ตัวเลข)
bad_amount = clean[clean["amount"].isna()] if "amount" in clean.columns else pd.DataFrame()
print("แถวที่ amount ว่าง/ไม่ใช่ตัวเลข:", len(bad_amount))
if len(bad_amount) > 0:
    display(bad_amount.head(10))


แถวที่ amount ว่าง/ไม่ใช่ตัวเลข: 34439


,year_fy,agency_code,agency_name,province,plan,output,activity_code,activity,budget_group,expense_type,sub_category,commitment_type,cap_ncap,item_code,item_name,amount,pp_code,output_kind,land_building,asset_band,subsidy_type,is_commitment,flagship_name,pur_commit_flag,_file,_sheet,รวม,nbud,รายการ,qty
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(ผู้พิมพ์รายงาน : PSIT วันที่ : 19 เมษายน 2564...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,แบบสรุปคำของบประมาณรายจ่ายประจำปีงบประมาณ พ.ศ....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN
59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1001_รายงานคำขอเบื้องต้น.xlsx,mask1_12,NaN,NaN,NaN,NaN


In [44]:
# 4.) ดูยอดรวมตัวอย่าง (คร่าว ๆ)
if "amount" in clean.columns:
    print("ยอดรวมทั้งหมด:", clean["amount"].sum())
    grp = ["_file","_sheet"] if "_file" in clean.columns else []
    if grp:
        print("\nTop 10 ยอดรวมต่อไฟล์/ชีต:")
        print(clean.groupby(grp)["amount"].sum().sort_values(ascending=False).head(10))


ยอดรวมทั้งหมด: 16097294274776.688

Top 10 ยอดรวมต่อไฟล์/ชีต:
_file                   _sheet                    
กรุงเทพ.xlsx            Table 1                       1.410071e+13
นนทบุรี.xlsx            Table 1                       1.996584e+12
1017_P1.xlsx            ข้อ 4                         8.576254e+03
                        ข้อ 5                         3.931491e+03
                        รายละเอียดลงทุน-ประจำ(กรม)    3.771730e+03
1001_รายการผูกพัน.xlsx  Sheet1                        5.572250e+02
1017_P1.xlsx            mask2                         2.100000e+02
                        mask3                         1.010000e+02
                        mask4                         1.010000e+02
1001_รายการผูกพัน.xlsx  Mask1                         6.000000e+01
Name: amount, dtype: float64


In [45]:
 # 5) หาหัวที่ยังไม่รู้จัก (จากไฟล์ unmapped)
unmap = pd.read_excel(out / "unmapped_columns.xlsx")
print("แถวหัวคอลัมน์ที่ยังไม่รู้จัก:", len(unmap))
if len(unmap):
    display(unmap.head(10))


แถวหัวคอลัมน์ที่ยังไม่รู้จัก: 4


,file,sheet,headers_snapshot
0,1017_P1.xlsx,หมายเหตุ,|
1,1017_P1.xlsx,mask1,bis62…|||||||||||||||
2,1017_P1.xlsx,Mask1_remake,bis62…|||||||||||||||
3,9012_ทะเบียนรายการ.xlsx,Mask1,||||
